In [1]:
import torch
from torch import nn
import torch.functional as F
import numpy as np

In [258]:
a = torch.arange(5)
a.tile((1, 2, 4)).split

tensor([[[0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4]]])

In [261]:
torch.eye(4).split([3, 1], dim=-1)


(tensor([[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.],
         [0., 0., 0.]]),
 tensor([[0.],
         [0.],
         [0.],
         [1.]]))

In [159]:
import collections
import torch


# Array of 3-component vectors, stored as individual array for
# each component.
Vecs = collections.namedtuple('Vecs', ['x', 'y', 'z'])

# Array of 3x3 rotation matrices, stored as individual array for
# each component.
Rots = collections.namedtuple('Rots', ['xx', 'xy', 'xz',
                                       'yx', 'yy', 'yz',
                                       'zx', 'zy', 'zz'])
# Array of rigid 3D transformations, stored as array of rotations and
# array of translations.
Rigids = collections.namedtuple('Rigids', ['rot', 'trans'])


def squared_difference(x, y):
    return torch.square(x - y)


def apply_tree(fun, *r: Rigids) -> Rigids:
    return Rigids(
        rot=Rots(*[fun(*[x.rot[i] for x in r]) for i in range(9)]),
        trans=Vecs(*[fun(*[x.trans[i] for x in r]) for i in range(3)])
    )

In [169]:
rig = Rigids(rot=Rots(*([np.ones((20, 5))] * 9)), trans=Vecs(*([np.ones((20, 5))] * 3)))

In [174]:
apply_tree(lambda x, y: np.concatenate([x[0], y[1]]), rig, rig)

Rigids(rot=Rots(xx=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), xy=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), xz=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), yx=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), yy=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), yz=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), zx=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), zy=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), zz=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])), trans=Vecs(x=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), y=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), z=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])))

In [107]:
t = torch.tensor([[10, 30, 20], [60, 40, 50]])
t.shape

torch.Size([2, 3])

In [214]:
mat = torch.tensor([0.8850, 0, 0, 0.4445, 0, 0, -0.1384, 0, 0]).reshape(3, 3)
torch.matmul(mat, torch.tensor((-0.520, 1.363, 0.000))) + torch.tensor([36.5560, 43.8110, -1.5430])

tensor([36.0958, 43.5799, -1.4710])

In [230]:
(mat < mat).type(mat.dtype)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [224]:
mat = torch.tensor(
   [[-0.8796,  0.3340, -0.3387],
          [-0.4209, -0.2145,  0.8814],
          [ 0.2217,  0.9179,  0.3293]]
)
torch.matmul(mat, mat.T)

tensor([[9.9997e-01, 5.0455e-05, 3.7394e-05],
        [5.0455e-05, 1.0000e+00, 4.1932e-05],
        [3.7394e-05, 4.1932e-05, 1.0001e+00]])

In [210]:
tmp = torch.arange(6)
tmp.tile(1, 2, 2)

tensor([[[0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5],
         [0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5]]])

In [195]:
(np.arange(4)[:, None] * mat).sum(0)

array([2., 1., 0., 3.])

In [197]:
mat = np.zeros((3, 4))
mat.swapaxes(0, 1).shape

(4, 3)

In [135]:
all_atom_positions = torch.arange(2 * 37 * 3).reshape(2, 37, 3)
residx_rigidgroup_base_atom37_idx = torch.tensor([
    [[1,2,3], [6,6,6]],
    [[7,7,9], [12,11,12]]
])
torch.gather(
    all_atom_positions.unsqueeze(-2).tile([1, 1, 3, 1]), 1, 
    residx_rigidgroup_base_atom37_idx.unsqueeze(-1).tile([1, 1, 1, 3])
)[1]



tensor([[[132, 133, 134],
         [132, 133, 134],
         [138, 139, 140]],

        [[147, 148, 149],
         [144, 145, 146],
         [147, 148, 149]]])

In [120]:
all_atom_positions.unsqueeze(-2).tile([1, 1, 2, 1]).shape

torch.Size([2, 37, 2, 3])

In [124]:
residx_rigidgroup_base_atom37_idx.unsqueeze(-1).shape #.tile([1, 1, 1, 2]).shape

torch.Size([2, 2, 3, 1])

In [125]:
residx_rigidgroup_base_atom37_idx.shape

torch.Size([2, 2, 3])

In [37]:
class Test(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.l = nn.Linear(10, 10)
        
    def forward(self, x):
        return {'a': self.l(x['a']), 'b': self.l(x['a'])}
    
    
model = nn.Sequential(Test(), Test(), Test())
model({'a': torch.zeros(5, 10), 'b': torch.zeros(5, 6, 10)})

{'a': tensor([[-0.0916,  0.0089,  0.1360,  0.3220,  0.1380,  0.3717,  0.1188, -0.3970,
          -0.2157, -0.2728],
         [-0.0916,  0.0089,  0.1360,  0.3220,  0.1380,  0.3717,  0.1188, -0.3970,
          -0.2157, -0.2728],
         [-0.0916,  0.0089,  0.1360,  0.3220,  0.1380,  0.3717,  0.1188, -0.3970,
          -0.2157, -0.2728],
         [-0.0916,  0.0089,  0.1360,  0.3220,  0.1380,  0.3717,  0.1188, -0.3970,
          -0.2157, -0.2728],
         [-0.0916,  0.0089,  0.1360,  0.3220,  0.1380,  0.3717,  0.1188, -0.3970,
          -0.2157, -0.2728]], grad_fn=<AddmmBackward>),
 'b': tensor([[-0.0916,  0.0089,  0.1360,  0.3220,  0.1380,  0.3717,  0.1188, -0.3970,
          -0.2157, -0.2728],
         [-0.0916,  0.0089,  0.1360,  0.3220,  0.1380,  0.3717,  0.1188, -0.3970,
          -0.2157, -0.2728],
         [-0.0916,  0.0089,  0.1360,  0.3220,  0.1380,  0.3717,  0.1188, -0.3970,
          -0.2157, -0.2728],
         [-0.0916,  0.0089,  0.1360,  0.3220,  0.1380,  0.3717,  0.1188, -0

In [70]:
model = Test()

In [74]:
inputs_1 = {'a': torch.zeros(5, 10), 'b': torch.zeros(5, 6, 10)}

with torch.no_grad():
    output_1 = model(inputs_1)
    
output_2 = model(output_1)
#inputs_2

In [75]:
output_1

{'a': tensor([[ 0.1672, -0.0184, -0.1907,  0.1110,  0.3153,  0.2358,  0.1021,  0.0036,
          -0.2361, -0.1048],
         [ 0.1672, -0.0184, -0.1907,  0.1110,  0.3153,  0.2358,  0.1021,  0.0036,
          -0.2361, -0.1048],
         [ 0.1672, -0.0184, -0.1907,  0.1110,  0.3153,  0.2358,  0.1021,  0.0036,
          -0.2361, -0.1048],
         [ 0.1672, -0.0184, -0.1907,  0.1110,  0.3153,  0.2358,  0.1021,  0.0036,
          -0.2361, -0.1048],
         [ 0.1672, -0.0184, -0.1907,  0.1110,  0.3153,  0.2358,  0.1021,  0.0036,
          -0.2361, -0.1048]]),
 'b': tensor([[ 0.1672, -0.0184, -0.1907,  0.1110,  0.3153,  0.2358,  0.1021,  0.0036,
          -0.2361, -0.1048],
         [ 0.1672, -0.0184, -0.1907,  0.1110,  0.3153,  0.2358,  0.1021,  0.0036,
          -0.2361, -0.1048],
         [ 0.1672, -0.0184, -0.1907,  0.1110,  0.3153,  0.2358,  0.1021,  0.0036,
          -0.2361, -0.1048],
         [ 0.1672, -0.0184, -0.1907,  0.1110,  0.3153,  0.2358,  0.1021,  0.0036,
          -0.2361,

In [76]:
output_2

{'a': tensor([[ 0.1213, -0.1375, -0.3580,  0.2230,  0.2943,  0.0403,  0.1334,  0.0267,
          -0.1734, -0.2460],
         [ 0.1213, -0.1375, -0.3580,  0.2230,  0.2943,  0.0403,  0.1334,  0.0267,
          -0.1734, -0.2460],
         [ 0.1213, -0.1375, -0.3580,  0.2230,  0.2943,  0.0403,  0.1334,  0.0267,
          -0.1734, -0.2460],
         [ 0.1213, -0.1375, -0.3580,  0.2230,  0.2943,  0.0403,  0.1334,  0.0267,
          -0.1734, -0.2460],
         [ 0.1213, -0.1375, -0.3580,  0.2230,  0.2943,  0.0403,  0.1334,  0.0267,
          -0.1734, -0.2460]], grad_fn=<AddmmBackward>),
 'b': tensor([[ 0.1213, -0.1375, -0.3580,  0.2230,  0.2943,  0.0403,  0.1334,  0.0267,
          -0.1734, -0.2460],
         [ 0.1213, -0.1375, -0.3580,  0.2230,  0.2943,  0.0403,  0.1334,  0.0267,
          -0.1734, -0.2460],
         [ 0.1213, -0.1375, -0.3580,  0.2230,  0.2943,  0.0403,  0.1334,  0.0267,
          -0.1734, -0.2460],
         [ 0.1213, -0.1375, -0.3580,  0.2230,  0.2943,  0.0403,  0.1334,  0

In [77]:
output_2['a'].sum().backward()

In [69]:
model.parameters()

TypeError: 'generator' object is not subscriptable

In [51]:
a = np.zeros((6,9))
a[cprod[:, 0], cprod[:, 1]] = 1
a

array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [2]:
a = torch.ones((2, 10, 5))
b = torch.ones((2, 4, 10, 5))
torch.einsum('bic,bmjc->bmij', a, b).shape

torch.Size([2, 4, 10, 10])

In [13]:
tmp = torch.einsum('bmix,bmjy->bmijxy', torch.ones((2, 1, 10, 5)), torch.ones((2, 3, 15, 5)))
tmp.shape

torch.Size([2, 3, 10, 15, 5, 5])

In [9]:
tmp.view(*tmp.shape[:-2], -1).shape

torch.Size([2, 10, 10, 25])

In [217]:
config = {
    'rec_profile': {
        'num_c': 64
    },
    'lig_profile': {
        'num_c': 64
    },
    'pair_rep': {
        'num_c': 64
    },
    'SeqRowAttentionWithPairBias': {
        'attention_num_c': 32,
        'num_heads': 8
    }
}


class SeqRowAttentionWithPairBias(nn.Module):
    def __init__(self, config):
        super().__init__()

        attn_num_c = config['SeqRowAttentionWithPairBias']['attention_num_c']
        num_heads = config['SeqRowAttentionWithPairBias']['num_heads']
        rec_num_c = config['rec_profile']['num_c']
        lig_num_c = config['lig_profile']['num_c']
        pair_rep_num_c = config['pair_rep']['num_c']
        self.rec_q = nn.Linear(rec_num_c, attn_num_c * num_heads, bias=False)
        self.rec_k = nn.Linear(rec_num_c, attn_num_c * num_heads, bias=False)
        self.rec_v = nn.Linear(rec_num_c, attn_num_c * num_heads, bias=False)
        self.lig_q = nn.Linear(lig_num_c, attn_num_c * num_heads, bias=False)
        self.lig_k = nn.Linear(lig_num_c, attn_num_c * num_heads, bias=False)
        self.lig_v = nn.Linear(lig_num_c, attn_num_c * num_heads, bias=False)

        self.rec_rec_project = nn.Linear(pair_rep_num_c, num_heads)
        self.lig_lig_project = nn.Linear(pair_rep_num_c, num_heads)
        self.rec_lig_project = nn.Linear(pair_rep_num_c, num_heads)
        
        self.rec_final = nn.Linear(attn_num_c * num_heads, rec_num_c)
        self.lig_final = nn.Linear(attn_num_c * num_heads, lig_num_c)
        
        self.attn_num_c = attn_num_c
        self.num_heads = num_heads

    def forward(self, rec_profile, lig_profile, rec_rec, lig_lig, rec_lig, lig_rec):
        batch_size = rec_profile.shape[0]
        num_res = rec_profile.shape[1]
        num_atoms = lig_profile.shape[2]
        num_cep = lig_profile.shape[1]
        
        rec_q = self.rec_q(rec_profile).view(*rec_profile.shape[:-1], self.attn_num_c, self.num_heads)
        rec_k = self.rec_k(rec_profile).view(*rec_profile.shape[:-1], self.attn_num_c, self.num_heads)
        rec_v = self.rec_v(rec_profile).view(*rec_profile.shape[:-1], self.attn_num_c, self.num_heads)
        lig_q = self.lig_q(lig_profile).view(*lig_profile.shape[:-1], self.attn_num_c, self.num_heads)
        lig_k = self.lig_k(lig_profile).view(*lig_profile.shape[:-1], self.attn_num_c, self.num_heads)
        lig_v = self.lig_v(lig_profile).view(*lig_profile.shape[:-1], self.attn_num_c, self.num_heads)
        
        weights = torch.zeros((batch_size, num_cep, num_res + num_atoms, num_res + num_atoms, self.num_heads), device=rec_profile.device, dtype=rec_profile.dtype)

        #print(rec_q.shape)
        #print(lig_k.shape)
        rec_rec_aff = torch.einsum('bich,bjch->bijh', rec_q, rec_k)
        rec_lig_aff = torch.einsum('bich,bmjch->bmijh', rec_q, lig_k)
        lig_rec_aff = torch.einsum('bich,bmjch->bmjih', rec_k, lig_q)
        lig_lig_aff = torch.einsum('bmich,bmjch->bmijh', lig_q, lig_k)

        rec_rec_bias = self.rec_rec_project(rec_rec).view(*rec_rec.shape[:-1], self.num_heads)
        lig_lig_bias = self.lig_lig_project(lig_lig).view(*lig_lig.shape[:-1], self.num_heads)
        rec_lig_bias = self.rec_lig_project(rec_lig).view(*rec_lig.shape[:-1], self.num_heads)
        lig_rec_bias = self.rec_lig_project(lig_rec).view(*lig_rec.shape[:-1], self.num_heads)
        
        #print(rec_rec_aff.shape)
        #print(rec_rec_bias.shape)
        print(weights.shape)
        weights[:, :, :num_res, :num_res] = rec_rec_aff + rec_rec_bias
        weights[:, :, :num_res, num_res:] = rec_lig_aff + rec_lig_bias
        weights[:, :, num_res:, :num_res] = lig_rec_aff + lig_rec_bias
        weights[:, :, num_res:, num_res:] = lig_lig_aff + lig_lig_bias
        weights = torch.softmax(weights, dim=-2)
        #print(weights)

        rec_profile = torch.einsum('brch,birh->bich', rec_v, weights[:, 0, :num_res, :num_res]) + torch.einsum('bmrch,bmirh->bmich', lig_v, weights[:, :, :num_res, num_res:]).mean(1)
        lig_profile = torch.einsum('bmrch,bmirh->bmich', lig_v, weights[:, :, num_res:, num_res:]) + torch.einsum('brch,bmirh->bmich', rec_v, weights[:, :, num_res:, :num_res])
        
        rec_profile = self.rec_final(rec_profile.reshape(*rec_profile.shape[:-2], -1))
        lig_profile = self.lig_final(lig_profile.reshape(*lig_profile.shape[:-2], -1))
        return rec_profile, lig_profile


In [218]:
layer = SeqRowAttentionWithPairBias(config)
#layer.to('cuda:0')


In [219]:
sum([torch.prod(torch.tensor(x.shape)) for x in layer.parameters()])

tensor(132760)

In [220]:
num_res = 100
num_atoms = 12
num_msa = 128
a = torch.ones(1, num_res, config['rec_profile']['num_c'])
b = torch.ones(1, num_msa, num_atoms, config['lig_profile']['num_c'])
rec_rec = torch.ones(1, num_res, num_res, config['pair_rep']['num_c'])
lig_lig = torch.ones(1, num_atoms, num_atoms, config['pair_rep']['num_c'])
rec_lig = torch.ones(1, num_res, num_atoms, config['pair_rep']['num_c'])
lig_rec = torch.ones(1, num_atoms, num_res, config['pair_rep']['num_c'])

In [221]:
a, b = layer(a, b, rec_rec, lig_lig, rec_lig, lig_rec)

torch.Size([1, 128, 112, 112, 8])
tensor([[[[[9.9821e-03, 9.6372e-03, 5.8093e-04,  ..., 9.9995e-03,
            9.0744e-03, 6.6431e-03],
           [9.9821e-03, 9.6372e-03, 5.8093e-04,  ..., 9.9995e-03,
            9.0744e-03, 6.6431e-03],
           [9.9821e-03, 9.6372e-03, 5.8093e-04,  ..., 9.9995e-03,
            9.0744e-03, 6.6431e-03],
           ...,
           [1.4957e-04, 3.0234e-03, 7.8492e-02,  ..., 4.1494e-06,
            7.7135e-03, 2.7974e-02],
           [1.4957e-04, 3.0234e-03, 7.8492e-02,  ..., 4.1494e-06,
            7.7135e-03, 2.7974e-02],
           [1.4957e-04, 3.0234e-03, 7.8492e-02,  ..., 4.1494e-06,
            7.7135e-03, 2.7974e-02]],

          [[9.9821e-03, 9.6372e-03, 5.8093e-04,  ..., 9.9995e-03,
            9.0744e-03, 6.6431e-03],
           [9.9821e-03, 9.6372e-03, 5.8093e-04,  ..., 9.9995e-03,
            9.0744e-03, 6.6431e-03],
           [9.9821e-03, 9.6372e-03, 5.8093e-04,  ..., 9.9995e-03,
            9.0744e-03, 6.6431e-03],
           ...,
     

In [209]:
b.shape

torch.Size([1, 128, 12, 64])

In [211]:
b.sum().backward()

RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling .backward() or autograd.grad() the first time.

In [73]:
layer.rec_k.

<generator object Module.parameters at 0x7f5874995cd0>

In [88]:
l = MultiheadLinear(10, 20, 4)

In [89]:
l

MultiheadLinear(in_features=10, out_features=20, num_heads=4, bias=True)

In [91]:
l(torch.ones(1,2,3,10)).shape

torch.Size([4, 20, 1, 2, 3])